In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from datetime import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from bci_dataset import BCIDataset

from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds
from brainflow.data_filter import DataFilter, FilterTypes, WindowOperations, NoiseTypes

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
# device = torch.device('cpu')

In [3]:
class BCIModel(nn.Module):
    def __init__(self, classes=3):
        super(BCIModel, self).__init__()
        
        self.conv1 = nn.Conv1d(8, 64, 1)
        self.conv2 = nn.Conv1d(64, 256, 1)
        self.conv3 = nn.Conv1d(256, 512, 1)
        self.conv4 = nn.Conv1d(512, 1024, 1)
        self.conv5 = nn.Conv1d(1024, 16, 1)
        
        self.fc1 = nn.Linear(16*64, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, classes)

    def forward(self, x):
        x = torch.sigmoid(self.conv1(x))
        x = torch.sigmoid(self.conv2(x))
        x = torch.sigmoid(self.conv3(x))
        x = torch.sigmoid(self.conv4(x))
        x = torch.sigmoid(self.conv5(x))
        
        x = torch.flatten(x, 1)
        
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return x


In [4]:
train_dataset = BCIDataset('raw_data_5min.npy')
test_dataset = BCIDataset('raw_data_test.npy')

train_dataloader = DataLoader(train_dataset, batch_size=2048, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=2048, shuffle=True)

/home/stelath/GitHub/neuro-nav/bci_dataset.py:40: ComplexWarning: Casting complex values to real discards the imaginary part
  inp[k] = DataFilter.perform_fft(channel[: hz // 2], WindowOperations.NO_WINDOW.value)[:hz // 4]


In [6]:
model = BCIModel()
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.BCELoss()

In [7]:
def fwd_pass(inputs, targets, train=False):
  inputs = inputs.to(device)
  targets = targets.to(device)

  if train:
    model.zero_grad()
  
  outputs = model(inputs)
  matches = [torch.argmax(i) == torch.argmax(j) for i, j in zip(outputs, targets)]
  acc = matches.count(True) / len(matches)
  loss = loss_function(outputs, targets)

  if train:
    loss.backward()
    optimizer.step()
  
  return acc, loss

In [8]:
now = datetime.now()
dt_string = now.strftime("%d:%m:%Y-%H:%M:%S")
writer = SummaryWriter(f'runs/BCI_{dt_string}')

EPOCHS = 60
global_step = 0
for epoch in range(EPOCHS):
    progress_bar = tqdm(total=len(train_dataloader))
    progress_bar.set_description(f"Epoch {epoch + 1}/{EPOCHS}")
    
    for step, batch in enumerate(train_dataloader):
        inputs = batch['inputs']
        targets = batch['targets']
        
        acc, loss = fwd_pass(inputs, targets, train=True)
        loss = loss.detach().item()
        
        progress_bar.update(1)
        logs = {"loss": loss, "accuracy": acc, "step": step}
        progress_bar.set_postfix(**logs)
        
        writer.add_scalar('Loss/train', loss, global_step)
        writer.add_scalar('Accuracy/train', acc, global_step)
        
        
        global_step += 1
        
    
    test_acc_list = []
    test_loss_list = []
    for batch in test_dataloader:
        inputs = batch['inputs']
        targets = batch['targets']
        
        test_acc, test_loss = fwd_pass(inputs, targets, train=False)
        
        test_acc_list.append(test_acc)
        test_loss_list.append(test_loss.detach().item())
    
    test_acc = np.mean(test_acc_list)
    test_loss = np.mean(test_loss_list)
    
    writer.add_scalar('Loss/test', test_loss, global_step)
    writer.add_scalar('Accuracy/test', test_acc, global_step)
    
    if (epoch + 1) % 5 == 0 or epoch + 1 == EPOCHS:
        torch.save(model.state_dict(), f"models/model_{epoch + 1}.pth")

  0%|          | 0/14 [00:00<?, ?it/s]

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.